# Ataques contra credenciales de usuario

En esta sección vamos a ver como se pueden realizar ataques contra cuentas de usuario y contraseña para intentar obtener unas credenciales válidas que o bien nos faciliten el acceso a un sistema remoto o bien que nos proporcionen acceso a una cuenta con más privilegios en el sistema o servicio que estemos auditando.

### Metodología

Se recomienda seguir usando [cherrytree](https://www.giuspen.com/cherrytree/) para ir guardando las evidencias.
1. <a style="color:#000;text-decoration:none;cursor:pointer" href="#intro_cuentas">Breve introducción informal a los ataques contra credenciales de usuario</a>
  * Tipos de ataques
  * Diccionarios
  * Credenciales por defecto
<br><br>
2. <a style="color:#000;text-decoration:none;cursor:pointer" href="#online">Ataques contra servicios online de forma activa</a>
  * Ataque de diccionario contra SSH usando patator
  * Ataque de diccionario contra SSH usando hydra
  * Ataque de diccionario contra FTP usando patator
  * Ataque de diccionario contra FTP usando hydra
  * Ataque de diccionario contra formularios web usando Burp Suite
<br><br>
3. <a style="color:#000;text-decoration:none;cursor:pointer" href="#almacenamiento_credenciales">Almacenamiento de credenciales</a>
  * Breve introducción informal al almacenamiento de credenciales en sistemas GNU/Linux
  * Obteniendo las credenciales en sistemas GNU/Linux (/etc/shadow)
  * Breve introducción informal al almacenamiento de credenciales en sistemas Windows
  * Breve introducción al framework Empire
  * Tomando el control de una máquina con Empire
  * Obteniendo los hashes de las contraseñas con el módulo credentials/powerdump de Empire
  * Obteniendo las credenciales en sistemas windows usando Empire
  * Obteniendo las contraseñas de memoria con mimikatz desde Empire
<br><br>
4. <a style="color:#000;text-decoration:none;cursor:pointer" href="#offline">Ataques offline - Cracking de contraseñas</a>
  * Identificar el algoritmo de hash usado para almacenar la contraseña
    * hash-identifier
    * hashid
  * Cracking de contraseñas con John The Ripper
  * Cracking de contraseñas con Hashcat
<br><br>

### Referencias usadas

1. Páginas web
  * [Empire](https://www.powershellempire.com/)

<a name="intro_cuentas"></a>

### Breve introducción informal a los ataques contra credenciales de usuario

Como las credenciales de usuario nos permiten acceder a un sistema, suelen ser un objetivo potencial para todas aquellas personas que deseen tener acceso, autorizado o no, al sistema. Por eso es importante que las credenciales de acceso y, específicamente, las contraseñas sean robustas. Es decir que, entre otros requisitos, cumplan:

  * Que combinen letras mayúsculas y minúsculas, números y símbolos como por ejemplo: .,:@/\|-_+\*
  * Tener una longitud mínima de 24 caracteres unicode
  * Tener una política de cambio de contraseña cada cierto tiempo
  * Asegurarse que no se utiliza una contraseña que ya haya sido utilizada anteriormente
  * Que sea una secuencia lo más aleatoria posible
  * Si no se utilizase una secuencia aleatoria -lo más recomendable-, pueden seguirse estrategias como utilizar la segunda letra de cada palabra de una frase intercalando números, letras mayúsculas,  minúsculas y símbolos.

#### Tipos de ataques

Existen distintos tipos de ataques que podemos realizar para conseguir las credenciales de un usuario. En muchas ocasiones, el usuario lo hemos conseguido en la fase de recopilación de información y sólo nos queda conseguir la contraseña. El temario especifíca 4 tipos de ataques que podemos realizar:

  * Ataques que no requieren conocimientos técnicos
    * Ingeniería social
  * Ataques a servicios online de forma activa
    * Ataques de diccionario
    * Ataques por fuerza bruta
  * Ataques a servicios online de forma pasiva
    * Realizando capturas de tráfico de red
  * Ataques offline
    * Cracking de contraseñas
    
#### Diccionarios

Si recordamos del tema de recopilación de información, en Kali Linux, por defecto, los diccionarios se guardan en la siguiente ruta:

<img src="img/kali_ruta_diccionarios.png" style="float:left">
<div style="clear:both"></div><br>

Uno de los más conocidos es *rockyou.txt*. Primero tenemos que descomprimirlo usando el comando gzip:

*gzip -d rockyou.txt.gz*

<img src="img/credenciales_rockyou_1.png" style="float:left">
<div style="clear:both"></div><br>

para poder usarlo:

<img src="img/credenciales_rockyou_2.png" style="float:left">
<div style="clear:both"></div><br>

Aparte de los que vienen incluidos en Kali, podemos encontrar otros en internet como los que proporciona [SecList](https://github.com/danielmiessler/SecLists):

<img src="img/seclist_diccionarios.png" style="float:left">
<div style="clear:both"></div><br>

#### Credenciales por defecto

Muchas empresas dejan configuradas credenciales por defecto en sus productos. Existen páginas que guardan estas credenciales por defecto como, por ejemplo, [CIRT](https://cirt.net/passwords):

<img src="img/credenciales_por_defecto_1.png" style="float:left">
<div style="clear:both"></div><br>

Este tipo de páginas pueden ser muy útiles cuando se realiza un ataque de diccionario.

<a name="online"></a>

### Ataques contra servicios online de forma activa 

En los ataques contra servicios que esten online vamos a ir probando credenciales hasta que demos con una que sea válida. Normalmente intentaremos conseguir algún usuario en la fase de recopilación de información y después intentar adivinar la contraseña usando un diccionario o por fuerza bruta. Aunque no se recomienda, si no encontrasemos un usuario válido, también podríamos realizar un ataque de diccionario o fuerza bruta para averiguar el usuario. Como vemos los modos de atacar son:

  * Fuerza bruta
  * Diccionario
  
El ataque por fuerza bruta prueba todas las [permutaciones](https://es.wikipedia.org/wiki/Permutación) posibles de los símbolos que le indiquemos hasta que de con el usuario o contraseña. Es un método muy lento a no ser que la contraseña sea muy corta. En un ataque de diccionario creamos una lista (diccionario) de palabras que creemos que pueden ser la contraseña. Existen muchos servicios online contra los que podemos lanzar un ataque. Algunos de ellos son:

  * SSH
  * FTP
  * Aplicaciones Web
  
#### Ataque de diccionario contra SSH usando patator

Tras descubrir un servicio SSH en la fase de recopilación de información:

<img src="img/credenciales_ssh_nmap.png" style="float:left">
<div style="clear:both"></div><br>

Vamos a ver como realizar un ataque de diccionario con el programa [patator](https://tools.kali.org/password-attacks/patator). Este programa ya le utilizamos para lanzar ataques de diccionario con el fin de enumerar los usuarios a través de SMTP. Para saber el módulo de patator que debemos utilizar, escribimos *patator* para listar los módulos y ver cuál es el módulo para ssh:

<img src="img/credenciales_ssh_patator_1.png" style="float:left">
<div style="clear:both"></div><br>

y una vez localizado el módulo, para saber como usarlo, escribimos *patator* seguido del nombre del módulo. En ese caso *patator ssh_login*:

<img src="img/credenciales_ssh_patator_2.png" style="float:left">
<div style="clear:both"></div><br>

Vemos que para poder lanzar el ataque debemos indicarle:

  * **host** => máquina a la que queremos atacar
  * **user** => el usuario para el que vamos a intentar adivinar la contraseña
  * **password** => para indicarle el listado (diccionario) de posibles contraseñas

Creamos una lista (diccionario) de posibles contraseñas usando cualquier editor de texto. En este caso usamos el editor vim:

<img src="img/credenciales_ssh_patator_3.png" style="float:left">
<div style="clear:both"></div><br>

para crear la lista:

<img src="img/credenciales_ssh_patator_4.png" style="float:left">
<div style="clear:both"></div><br>

y lanzamos patator usando el usuario *msfadmin* que descubrimos en la fase de recopilación de información. Sabiendo que la máquina metasploitable 2 esta en la dirección (192.168.119.8), el commando es:

*patator ssh_login host=192.168.119.8 user=msfadmin password=FILE0 0=contraseñas_a_comprobar.txt*

<img src="img/credenciales_ssh_patator_5.png" style="float:left">
<div style="clear:both"></div><br>

Vemos que *patator* va comprobando todas las contraseñas y nos informa de la que funciona. En este caso *msfadmin*. Como hicimos al enumerar a través del protocolo SMTP, podemos ignorar las contraseñas no válidas usar la opción *-x ignore:*. En este caso usariamos *-x ignore:mesg='Authentication failed.'*. El comando completo es:

*patator ssh_login host=192.168.119.8 user=msfadmin password=FILE0 0=contraseñas_a_comprobar.txt -x ignore:mesg='Authentication failed.'*

<img src="img/credenciales_ssh_patator_6.png" style="float:left">
<div style="clear:both"></div><br>

y como vemos sólo nos mostraría la contraseña válida. Podemos comprobar que la contraseña es válida conectandonos por ssh a la máquina usando el usuario *msfadmin*:

<img src="img/credenciales_ssh_patator_7.png" style="float:left">
<div style="clear:both"></div><br>

y la contraseña *msfadmin*:

<img src="img/credenciales_ssh_patator_8.png" style="float:left">
<div style="clear:both"></div><br>

Una vez conectados podemos ejecutar algún comando como *whoami*:

<img src="img/credenciales_ssh_patator_9.png" style="float:left">
<div style="clear:both"></div><br>

#### Ataque de diccionario contra SSH usando hydra

Vamos a ver como realizar el mismo ataque usando el programa [hydra](https://tools.kali.org/password-attacks/hydra). Para saber cómo utilizarlo, hacemos lo mismo que hicimos con el programa patator. Tecleamos *hydra* y pulsamos intro para que nos muestre algún ejemplo:

<img src="img/credenciales_ssh_hydra_1.png" style="float:left">
<div style="clear:both"></div><br>

vemos que algunas de las opciones interesantes son:

  * **-l {username}** para pasarle el usuario
  * **-L {ficheroConUsuariosAProbar}** para pasarle un diccionario (fichero) con los usuarios que queramos comprobar
  * **-p {contraseña}** para pasarle el usuario
  * **-P {ficheroConContraseñasAProbar}** para pasarle un diccionario (fichero) con las contraseñas que queramos comprobar
  * **-s {NúmeroDePuerto}** para indicarle el puerto en el que esta el servicio contra el que queremos lanzar el ataque
  
En este caso, realizaremos un ataque de diccionario contra el servicio ssh. Usaremos el usuario *msfadmin* y le pasaremos un fichero con las posibles contraseñas:

*hydra -l msfadmin -P contraseñas_a_comprobar.txt ssh://192.168.119.8*

<img src="img/credenciales_ssh_hydra_2.png" style="float:left">
<div style="clear:both"></div><br>

Vemos que nos muestra el usuario y contraseña correcta.

#### Ataque de diccionario contra FTP usando patator

Patator puede usarse para lanzar ataques de diccionario contra multiples servicios. En la fase de recopilación de información también encontramos 2 servidores [FTP](https://tools.ietf.org/html/rfc959):

<img src="img/credenciales_ftp_nmap.png" style="float:left">
<div style="clear:both"></div><br>

Vamos a realizar un ataque de diccionario contra el servicio FTP en el puerto 21. En este caso vamos a usar 2 diccionarios:

  * uno para los usuarios
  * otro para las contraseñas

El diccionario (fichero) de contraseñas ya lo tenemos creado. Creamos uno de usuarios usando cualquier editor de texto:

<img src="img/credenciales_ftp_vim_1.png" style="float:left">
<div style="clear:both"></div><br>

y añadimos los usuarios que queramos comprobar:

<img src="img/credenciales_ftp_vim_2.png" style="float:left">
<div style="clear:both"></div><br>

Una vez creados el diccionario, para saber el módulo de patator que debemos utilizar, escribimos *patator* para listar los módulos y ver cuál es el módulo para ftp:

<img src="img/credenciales_ftp_patator_1.png" style="float:left">
<div style="clear:both"></div><br>

y una vez localizado el módulo, para saber como usarlo, escribimos *patator* seguido del nombre del módulo. En ese caso *patator ftp_login*:

<img src="img/credenciales_ftp_patator_2.png" style="float:left">
<div style="clear:both"></div><br>

Vemos que el modo de uso es, prácticamente, igual al del módulo de ssh. Para poder lanzar el ataque debemos indicarle:

  * **host** => máquina a la que queremos atacar
  * **user** => para indicarle el listado (diccionario) de posibles usuarios
  * **password** => para indicarle el listado (diccionario) de posibles contraseñas
  
El comando es:

*patator ftp_login host=192.168.119.8 user=FILE0 password=FILE1 0=users.txt 1=contraseñas_a_comprobar.txt*

<img src="img/credenciales_ftp_patator_3.png" style="float:left">
<div style="clear:both"></div><br>

Vemos como patator va comprobando cada una de las contraseñas para cada usuario y vemos que, el usuario *msfadmin* y la contraseña *msfadmin* son credenciales válidas para acceder al servidor FTP. Como en ejemplo anteriores, para filtrar los resultados de patator podemos usar la opción *-x ignore*. Por ejemplo, le podemos decir que ignore los códigos 530 ya que son intentos de login fallidos:

*patator ftp_login host=192.168.119.8 user=FILE0 password=FILE1 0=users.txt 1=contraseñas_a_comprobar.txt -x ignore:code=530*

<img src="img/credenciales_ftp_patator_4.png" style="float:left">
<div style="clear:both"></div><br>

Una vez obtenidas las credenciales podemos comprobar que tenemos acceso al servicio. Para ello, por ejemplo, podemos usar el programa *telnet*:

<img src="img/credenciales_ftp_login_1.png" style="float:left">
<div style="clear:both"></div><br>

Le indicamos nuestro usuario con el comando *USER*:

<img src="img/credenciales_ftp_login_2.png" style="float:left">
<div style="clear:both"></div><br>

A continuación le indicamos la contraseña con el comando *PASS*:

<img src="img/credenciales_ftp_login_3.png" style="float:left">
<div style="clear:both"></div><br>

y ya estamos dentro:

<img src="img/credenciales_ftp_login_4.png" style="float:left">
<div style="clear:both"></div><br>

#### Ataque de diccionario contra FTP usando hydra

También podemos realizar el mismo ataque usando hydra. En este caso, también le pasamos a hydra el fichero de usuarios (users.txt) y fichero de contraseñas (contraseñas_a_comprobar.txt):

*hydra -L users.txt -P contraseñas_a_comprobar.txt ftp://192.168.119.8*

<img src="img/credenciales_ftp_hydra.png" style="float:left">
<div style="clear:both"></div><br>

En el resultado vemos que nos vuelve a encontrar al usuario *msfadmin* y la contraseña *msfadmin*.

#### Ataque de diccionario contra formularios web usando Burp Suite

Tras descubrir un servidor web en la fase de recopilación de información:

<img src="img/credenciales_http_nmap.png" style="float:left">
<div style="clear:both"></div><br>

Accedemos a el servicio usando un navegador web como Firefox y hacemos click sobre *DVWA*:

<img src="img/credenciales_http_dvwa_1.png" style="float:left">
<div style="clear:both"></div><br>

Nos logamos en la aplicación con las siguientes credenciales:

  * **Username:** admin
  * **Password:** password

<img src="img/credenciales_http_dvwa_2.png" style="float:left">
<div style="clear:both"></div><br>

A continuación hacemos click en el botón *DVWA Security* de la barra lateral izquierda:

<img src="img/credenciales_http_dvwa_3.png" style="float:left">
<div style="clear:both"></div><br>

Seleccionamos *Low* en el desplegable:

<img src="img/credenciales_http_dvwa_4.png" style="float:left">
<div style="clear:both"></div><br>

y pulsamos el botón *Submit*:

<img src="img/credenciales_http_dvwa_5.png" style="float:left">
<div style="clear:both"></div><br>

Ahora hacemos click en el botón *Brute Force* en la barra lateral izquierda:

<img src="img/credenciales_http_dvwa_6.png" style="float:left">
<div style="clear:both"></div><br>

Para ver el formulario sobre el que haremos el ataque de diccionario:

<img src="img/credenciales_http_dvwa_7.png" style="float:left">
<div style="clear:both"></div><br>

Una vez en la página del formulario, vamos a seguir los mismos pasos que en el capítulo de recopilación de información para activar [Burp Suite](https://portswigger.net/burp/), para ello, desplegamos el menú de opciones en Firefox y seleccionamos *Preferences*:

<img src="img/credenciales_http_burp_1.png" style="float:left">
<div style="clear:both"></div><br>

A continuación pulsamos el botón *Settings...* dentro de la pestaña Network de la sección Advanced:

<img src="img/credenciales_http_burp_2.png" style="float:left">
<div style="clear:both"></div><br>

Cumplimentamos las opciones como se ve en la siguiente imagen y pulsamos el botón *OK*:

<img src="img/credenciales_http_burp_3.png" style="float:left">
<div style="clear:both"></div><br>

Después ejecutamos *burp*:

<img src="img/credenciales_http_burp_4.png" style="float:left">
<div style="clear:both"></div><br>

Pulsamos el botón *Next* para continuar:

<img src="img/credenciales_http_burp_5.png" style="float:left">
<div style="clear:both"></div><br>

Finalmente pulsamos el botón *Start Burp* para iniciar Burp Suite:

<img src="img/credenciales_http_burp_6.png" style="float:left">
<div style="clear:both"></div><br>

Cuando se inicie burp, pulsamos en la pestaña *Proxy* y después en la pestaña *Options*. En las opciones activamos "Incercept responses based on the following rules" para que que el proxy de burp intercepte las respuestas http del servidor antes de mostrarlas en el navegador:

<img src="img/credenciales_http_burp_7.png" style="float:left">
<div style="clear:both"></div><br>

Después hacemos click en la pestaña *Intercept* y nos aseguramos que el botón tiene el texto:

*Intercept is on*

<img src="img/credenciales_http_burp_8.png" style="float:left">
<div style="clear:both"></div><br>

Volvemos a Firefox. Metemos el usuario y contraseña que queramos en formulario y pulsamos el botón *Login*:

<img src="img/credenciales_http_burp_9.png" style="float:left">
<div style="clear:both"></div><br>

Volvemos a burp y veremos que la petición http que ha hecho el navegador ha sido capturada. Pulsamos el botón *Forward* para continuar:

<img src="img/credenciales_http_burp_10.png" style="float:left">
<div style="clear:both"></div><br>

Volvemos a hacer click en el botón *Forward*:

<img src="img/credenciales_http_burp_11.png" style="float:left">
<div style="clear:both"></div><br>

para que la respuesta del servidor llegue a nuestro navegador:

<img src="img/credenciales_http_burp_12.png" style="float:left">
<div style="clear:both"></div><br>

Vemos que las credenciales no existen -a no ser que hayamos tenido mucha suerte- y nos aparece el mensaje de login failed. Vamos a usar burp intruder para lanzar un ataque de diccionario contra el usuario y la contraseña. Para ello, seleccionamos la petición HTTP que nos ha dado la respuesta del servidor en la pestaña *HTTP history* y hacemos click con el botón derecho del ratón para desplegar el menú de opciones. Seleccionamos la opción *Send to Intruder*:

<img src="img/credenciales_http_burp_13.png" style="float:left">
<div style="clear:both"></div><br>

Hacemos click en la Pestaña Intruder y después en la pestaña *Positions*:

<img src="img/credenciales_http_burp_14.png" style="float:left">
<div style="clear:both"></div><br>

Seleccionamos *Cluster bomb* en el menú desplegable *Attack Type*:

<img src="img/credenciales_http_burp_15.png" style="float:left">
<div style="clear:both"></div><br>

Pulsamos el botón *Clear*:

<img src="img/credenciales_http_burp_16.png" style="float:left">
<div style="clear:both"></div><br>

para desmarcar todos los valores de los parámetros marcados en naranja:

<img src="img/credenciales_http_burp_17.png" style="float:left">
<div style="clear:both"></div><br>

Ahora seleccionaremos los valores de los parámetros sobre los que queremos hacer fuerza bruta. Hacemos doble click sobre el valor del parámetro username y pulsamos el botón *Add*:

<img src="img/credenciales_http_burp_18.png" style="float:left">
<div style="clear:both"></div><br>

A continuación, hacemos doble click en el parámetro password y colvemos a pulsar el botón *Add*:

<img src="img/credenciales_http_burp_19.png" style="float:left">
<div style="clear:both"></div><br>

Una vez añadidos los parámetros, pulsamos el botón *Payloads*:

<img src="img/credenciales_http_burp_20.png" style="float:left">
<div style="clear:both"></div><br>

Dejamos las opciones que vienen por defecto en la sección * Payload Sets*:

  * **Payload set:** 1
  * **Payload type:** Simple list
  
y en la sección *Payload Options [Simple list]* vamos añadiendo usuarios a la lista. Escribimos un usuario y hacemos click en el botón *Add*:

<img src="img/credenciales_http_burp_21.png" style="float:left">
<div style="clear:both"></div><br>

Hacemos lo mismo por cada usuario que queramos añadir. Por ejemplo, añadimos otro usuario que se llame admin:

<img src="img/credenciales_http_burp_22.png" style="float:left">
<div style="clear:both"></div><br>

Una vez especificados los usuarios que queremos probar, seleccionamos el número 2 en el menú desplegable *Payload type:* para indicar las contraseñas que queremos probar para cada usuario que hemos definido en el punto anterior:

<img src="img/credenciales_http_burp_23.png" style="float:left">
<div style="clear:both"></div><br>

En este caso, en vez ir añadiendo las contraseñas que queremos probar una a una, utilizaremos un fichero (diccionario) con las contraseñas a probar. Para ello, seleccionamos *Runtime file* del menú desplegable *Payload type*:

<img src="img/credenciales_http_burp_24.png" style="float:left">
<div style="clear:both"></div><br>

Ahora pulsaremos el botón *Select File...* para seleccionar el fichero (diccionario) donde tenemos la lista de contraseñas:

<img src="img/credenciales_http_burp_25.png" style="float:left">
<div style="clear:both"></div><br>

Seleccionamos el fichero y pulsamos el botón *Open*:

<img src="img/credenciales_http_burp_26.png" style="float:left">
<div style="clear:both"></div><br>

Una vez seleccionado el fichero, pulsamos sobre la pestaña *Options*:

<img src="img/credenciales_http_burp_27.png" style="float:left">
<div style="clear:both"></div><br>

y en la sección *Grep - Extract*  activamos la opción *Extract the following from responses:* y pulsamos el botón *Add*:

<img src="img/credenciales_http_burp_28.png" style="float:left">
<div style="clear:both"></div><br>

Seleccionamos el texto de error que muestra cuando las credenciales son incorrectas y pulsamos el botón *OK*:

<img src="img/credenciales_http_burp_29.png" style="float:left">
<div style="clear:both"></div><br>

Ya podemos lanzar el ataque de diccionario contra el usuario y la contraseña pulsando el botón *Start Attack*:

<img src="img/credenciales_http_burp_30.png" style="float:left">
<div style="clear:both"></div><br>

Si tenemos la versión gratuita de burp nos aparecerá el siguiente mensaje. Pulsamos el botón *OK*:

<img src="img/credenciales_http_burp_31.png" style="float:left">
<div style="clear:both"></div><br>

y vemos como va probando todas las contraseñas para cada uno de los usuarios y, en la única donde no hay mensaje de error, es la que nos indica el usuario y contraseña correcto:

<img src="img/credenciales_http_burp_32.png" style="float:left">
<div style="clear:both"></div><br>

<a name="almacenamiento_credenciales"></a>

### Almacenamiento de credenciales

En esta sección vamos a coger las credenciales para poder en la siguiente sección crackearlas. A no ser qué tengamos mucha suerte y podamos obtener las contraseñas por medios públicos o mediante ingeniería social, en la mayoría de las ocasiones, podremos obtener las contraseñas de todos los usuarios de un sistema cuando hayamos accedido al sistema y hayamos escalado privilegios. En esta sección se supone que ya hemos accedido al sistema y tenemos el máximo privilegio. Vamos a ver cómo obtener las contraseñas en sistemas GNU/Linux y en sistemas Windows.

#### Breve introducción informal al almacenamiento de credenciales en sistemas GNU/Linux

Aunque en algún sistema GNU/Linux antiguo, las contraseñas se guardasen en texto plano en el fichero *passwd* que podemos encontrar en la ruta: */etc/passwd*. En los sistemas GNU/Linux actuales, los usuarios se guardan en ese fichero passwd (*/etc/passwd*):

<img src="img/credenciales_linux_1.png" style="float:left">
<div style="clear:both"></div><br>

y los hashes de las contraseñas se guardan en un fichero que se llama shadow que podemos encontrar en la siguiente ruta: */etc/shadow*. Para leer el fichero shadow (*/etc/shadow*) se necesitan permisos de superusuario. En este caso visualizamos las últimas 9 líneas del fichero:

<img src="img/credenciales_linux_2.png" style="float:left">
<div style="clear:both"></div><br>

Como podremos observar hay usuarios que tienen hashes y otros que no. Lo usuarios con hashes son los que pueden iniciar sessión y los otros suelen ser servicios que necesitan un usuario para funcionar. Para saber qué algoritmo de hashing se esta utilizando para almacenar la contraseña, podemos consultar el fichero *login.defs* que está en la ruta: */etc/login.defs*. En este caso, para no leer el fichero entero, lo filtramos usando el comando *grep* con las siguientes opciones:

  * **-i** para que busque coincidencias sin importar si las letras son mayúsculas o minúsculas
  * **-P** para usar expresiones regulares de tipo [PCRE](https://es.wikipedia.org/wiki/Perl_Compatible_Regular_Expressions). En este caso vamos a usar el símbolo ^ que indica que la línea debe comenzar por la palabra que indiquemos tras el símbolo ^
  
*cat /etc/login.defs | grep -iP "^encrypt_method"*

<img src="img/credenciales_linux_3.png" style="float:left">
<div style="clear:both"></div><br>

Vemos que se esta usando *SHA512*

#### Obteniendo las credenciales en sistemas GNU/Linux (/etc/shadow)

Como no podemos copiarnos directamente el fichero */etc/shadow*, vamos a realizar una copia de ese fichero y después nos lo descargaremos. Primero lo descargaremos de la máquina metasploitable 2 que es una distribución de GNU/Linux antigua y después los descargaremos de la máquina virtual de ubuntu server 16.04.3 LTS. Ya que tenemos las claves de acceso para la máquina metasploitable, nos conectamos por ssh:

<img src="img/credenciales_metasploitable_1.png" style="float:left">
<div style="clear:both"></div><br>

Una vez dentro, copiamos el contenido a un fichero para poder descargarnoslo desde Kali Linux. Para ello ejecutamos el comando *sudo* para lanzar el comando *cat* con privilegios de superusuario y poder leer el fichero */etc/shadow*. El contenido del fichero lo guardamos en un fichero de texto:

*sudo cat /etc/shadow > contraseñas_metasploitable2*

<img src="img/credenciales_metasploitable_2.png" style="float:left">
<div style="clear:both"></div><br>

Una vez creado el fichero, salimos de la sesión ssh usando el comando *exit*:

<img src="img/credenciales_metasploitable_3.png" style="float:left">
<div style="clear:both"></div><br>

Ahora podremos descargar el fichero usando, por ejemplo, el programa scp. Para ello debemos indicarle como primer argumento la ruta donde está el fichero que queremos descargar. En este caso queremos el fichero *contraseñas_metasploitable2* que está en la carpeta home del usuario *msfadmin*. Como segundo argumento debemos indicar dónde queremos copiarlo. En este caso la carpeta actual. Por lo que ejecutaríamos el comando de la siguiente forma:

*scp msfadmin@192.168.119.8:~/contraseñas_metasploitable2 .*

<img src="img/credenciales_metasploitable_4.png" style="float:left">
<div style="clear:both"></div><br>

Comprobamos que se ha descargado correctamente, leyendo las primeras 13 líneas del archivo:

<img src="img/credenciales_metasploitable_5.png" style="float:left">
<div style="clear:both"></div><br>

Una vez descargado el fichero, podemos borrarlo de metasploitable:

<img src="img/credenciales_metasploitable_6.png" style="float:left">
<div style="clear:both"></div><br>

Para descargar los hashes de la máquina virtual de ubuntu server, creamos un fichero con el contenido del fichero */etc/shadow*:

<img src="img/credenciales_ubuntu_1.png" style="float:left">
<div style="clear:both"></div><br>

Copiamos ese fichero a la máquina metasploitable 2 usando scp (secure copy):

<img src="img/credenciales_ubuntu_2.png" style="float:left">
<div style="clear:both"></div><br>

Finalmente, descargamos el fichero en Kali Linux:

<img src="img/credenciales_ubuntu_3.png" style="float:left">
<div style="clear:both"></div><br>

y, como en el caso anterior, una vez descargado el fichero, lo borramos de metasploitable:

<img src="img/credenciales_ubuntu_4.png" style="float:left">
<div style="clear:both"></div><br>

#### Breve introducción informal al almacenamiento de credenciales en sistemas Windows

**Pendiente de desarrollar**
Las constraseñas en windows se almacenan en el fichero (base de datos) SAM.

#### Breve introducción al framework Empire 

[Empire](http://www.powershellempire.com/) es un framework, que usa python y powershell, que nos va a ser útil tras haber ganado acceso a un sistema. Algunas partes importantes del framework son:

  * **listeners** Nos van a permitir poner un servicio a la escucha en un puerto determinado para que otro equipo (víctima) se conecte a él. Por ejemplo, en un sistema windows que hayamos comprometido ejecutamos un comando en powershell (launcher/stager) para que se conecte al *listener* que hemos dejado a la escucha.
  * **stagers** Son un conjunto de macros, librerias dinámicas y scripts que podemos utilizar para establecer conexión con el equipo comprometido. Por ejemplo, un script de powershell que conecte la máquina en la que se ejecute a un *listener*
  * **agents** Son las máquinas comprometidas de las que tenemos el control
  * **modules** Son todos los comandos y scripts que se pueden ejecutar en la máquina comprometida. Por ejemplo hay scripts que realizan capturas de pantalla o que pueden encender una cámara web.
  * **scripts** Son los scripts que podemos crear nosotros para expandir el framework

El primer paso es instalarlo en Kali Linux. Para ello accedemos al repositorio del framework [Empire](https://github.com/EmpireProject/Empire) en github. Pulsamos sobre el botón verde con letras blancas *Clone or download* y copiamos la url:

<img src="img/empire_1.png" style="float:left">
<div style="clear:both"></div><br>

Clonamos el repositorio:

<img src="img/empire_2.png" style="float:left">
<div style="clear:both"></div><br>

Accedemos a la carpeta *Empire*:

<img src="img/empire_3.png" style="float:left">
<div style="clear:both"></div><br>

Después accedemos a la carpeta *setup*:

<img src="img/empire_4.png" style="float:left">
<div style="clear:both"></div><br>

y lo instalamos ejecutando el fichero *install.sh*:

<img src="img/empire_5.png" style="float:left">
<div style="clear:both"></div><br>

La instalación se pausará cuando llegue a la frase en amarillo. Pulsamo intro para continuar:

<img src="img/empire_6.png" style="float:left">
<div style="clear:both"></div><br>

Una vez instalado, volvemos un directorio atrás:

<img src="img/empire_7.png" style="float:left">
<div style="clear:both"></div><br>

Ejecutamos empire:

<img src="img/empire_8.png" style="float:left">
<div style="clear:both"></div><br>

Tras iniciarse, veremos el promtp de Empire:

<img src="img/empire_9.png" style="float:left">
<div style="clear:both"></div><br>

Para ver la lista de comandos que podemos ejecutar, podemos usar el comando *help*:

<img src="img/empire_10.png" style="float:left">
<div style="clear:both"></div><br>

#### Tomando el control de una máquina con Empire

Lo primero que vamos a hacer es crear un *listener* para crear un servicio que este a la escucha para que la máquina que comprometamos se pueda conectar a ese servicio. Para ello, ejecutamos el comando *listeners*. Veremos que el prompt cambia y entramos en las opciones para listeners:

<img src="img/empire_11.png" style="float:left">
<div style="clear:both"></div><br>

Si quisieramos volver al menú anterior, podemos ejecutar el comando *back* en cualquier momento. Desde el prompt de listeners, ejecutamos tecleamos *help* y pulsamos intro para ver todos los comandos que podemos ejecutar relativos a los *listeners*:

<img src="img/empire_12.png" style="float:left">
<div style="clear:both"></div><br>

Para crear un listener (servicio a la escucha), usamos el comando *uselistener*. Vamos a crear un servicio web a la escucha a través del protocolo *http*. Para ello, usamos el comando:

*uselistener http*

<img src="img/empire_13.png" style="float:left">
<div style="clear:both"></div><br>

Vemos que el promtp cambia de nuevo. Para ver las opciones que podemos configurar del servicio que pondremos a la escucha, podemos ejecutar el comando *info* para que nos muestre la información:

<img src="img/empire_14.png" style="float:left">
<div style="clear:both"></div><br>

Para cambiar las opciones, usaremos el comando *set*. Por ejemplo, cambiamos de nombre al listener usando:

*set Name credencialesWindows*

<img src="img/empire_15.png" style="float:left">
<div style="clear:both"></div><br>

Si volvemos a ejecutar *info* veremos que el nombre ha cambiado:

<img src="img/empire_16.png" style="float:left">
<div style="clear:both"></div><br>

Una vez que hemos configurado las opciones que queramos, lo ejecutamos con el comando *execute*. En este caso, salvo el nombre del servicio, hemos dejado todo por defecto:

<img src="img/empire_17.png" style="float:left">
<div style="clear:both"></div><br>

Podemos ver que el listener esta a la escucha ejecutando el comando *listeners*:

<img src="img/empire_18.png" style="float:left">
<div style="clear:both"></div><br>

Una vez que tenemos el listener (servicio) a la escucha, tenemos que crear un *launcher*. Un *launcher* es un script que vamos a crear para ejecutarlo en la máquina que queremos que se conecte al servicio (listener) que tenemos a la escucha. Los launchers podemos crearlos en python o en powershell. En este caso, como vamos a usarlo en una máquina windows, usamos powershell. Para crear el launcher debemos usar el comando *launcher* seguido del lenguaje de scripting (python o powershell) y el nombre del servicio al que queremos que se conecte. En este caso:

*launcher powershell credencialesWindows*

<img src="img/empire_19.png" style="float:left">
<div style="clear:both"></div><br>

Copiamos el script y, por ejemplo, en la máquina de windows 10 abrimos el interprete de comandos con permisos de administrador:

<img src="img/empire_20.png" style="float:left">
<div style="clear:both"></div><br>

Aceptamos la advertencia:

<img src="img/empire_21.png" style="float:left">
<div style="clear:both"></div><br>

Ejecutamos el comando de powershell:

<img src="img/empire_22.png" style="float:left">
<div style="clear:both"></div><br>

y en Kali Linux veremos que se conecta a nuestro listener (servicio):

<img src="img/empire_23.png" style="float:left">
<div style="clear:both"></div><br>

Pulsamos intro y ejecutamos el comando *back* para volver al menú principal:

<img src="img/empire_24.png" style="float:left">
<div style="clear:both"></div><br>

para volver a la pantalla principal donde podremos ver que tenemos un agent activo:

<img src="img/empire_25.png" style="float:left">
<div style="clear:both"></div><br>

Si recordáis de la introducción un agent es una máquina que esta bajo control. El script de powershell ejecutado en la máquina windows 10, está en memoria, lo que lo hace mucho más difícil de detectar que si hubiese creado algún fichero. Ahora que hemos tomado el control de la máquina (agent), vamos a ver cómo interactuar con ella y ejecutar comandos

#### Obteniendo los hashes de las contraseñas con el módulo credentials/powerdump de Empire

Ejecutamos el comando *agents* para ver los agentes (máquinas bajo nuestro control) que tenemos y poder acceder a interactuar con ellos. Vemos que aparte de mostrarnos la información, el prompt cambia:

<img src="img/empire_26.png" style="float:left">
<div style="clear:both"></div><br>

El número de proceso (5724) de powershell es todo lo que el usuario verá si lista las tareas en el equipo. Podemos cambiar el nombre que asigna automáticamente al agente por otro que nos sea más fácil recordar usando el comando rename:

*rename GEU9T3MP win10creds*

<img src="img/empire_27.png" style="float:left">
<div style="clear:both"></div><br>

Para interactuar con la máquina comprometida (agente), utilizamos el comando *interact* seguido del nombre del agente:

<img src="img/empire_28.png" style="float:left">
<div style="clear:both"></div><br>

Para ver todos los comandos que podemos ejecutar en el agente, usamos el comando help:

<img src="img/empire_29.png" style="float:left">
<div style="clear:both"></div><br>

Para extraer los hashes de las contraseñas vamos a usar el modulo de powershell *credentials/powerdump*. Si recordáis de la introducción, los *modules* son todos los comandos y scripts -en este caso vamos a usar uno de powershell- que podemos utilizar en el agente (máquina comprometida). Para usar un módulo, tecleamos el comando *usemodule* seguido del nombre del módulo que queramos usar. En este caso:

*usemodule credentials/powerdump*

<img src="img/empire_30.png" style="float:left">
<div style="clear:both"></div><br>

Podemos ver la información del módulo usando el comando *info*. Entre las opciones hay una a la que debemos prestar atención que es *OpsecSafe*. Si el valor de esta opción es *True*, significa que el usuario no tendrá constancia que hemos ejecutado el comando:

<img src="img/empire_31.png" style="float:left">
<div style="clear:both"></div><br>

Para ejecutar el módulo, teclamos *execute* y pulsamos intro. Vemos que nos muestra los hashes de todos los usuarios:

<img src="img/empire_32.png" style="float:left">
<div style="clear:both"></div><br>

Estos hashes son los que en la siguiente sección crackearemos.

#### Obteniendo las contraseñas de memoria con mimikatz desde Empire

En vez hashes, lo que se suele intentar es extraer de memoria las contraseñas usando *mimikatz* desde Empire. En sistemas operativos anteriores a windows 10, en muchas ocasiones podremos extraer las contraseñas en texto plano, pero en windows 10, sólo obtendremos los hashes. ·Ejecutamos el comando *back* para salir del módulo en el que estamos:

<img src="img/empire_33.png" style="float:left">
<div style="clear:both"></div><br>

Tecleamos el comando *mimikatz* para ejecutarlo:

<img src="img/empire_34.png" style="float:left">
<div style="clear:both"></div><br>

Una vez hayamos terminado de interactuar con el agente (máquina comprometida), podemos ejecutar el comando *back* para volver al prompt de *agents*:

<img src="img/empire_35.png" style="float:left">
<div style="clear:both"></div><br>

Cuando queramos liberar un agente (máquina comprometida), desde el prompt usaremos el comando *remove* seguido del nombre del agente a borrar:

<img src="img/empire_36.png" style="float:left">
<div style="clear:both"></div><br>

Una vez borrado, veremos que si listamos los agentes con el comando *list* ha desaparecido:

<img src="img/empire_37.png" style="float:left">
<div style="clear:both"></div><br>

Una vez liberados todos *agents* (máquinas comprometidas) conectados al servicio (listener) http que creamos, podemos también eliminar el *listener* (servicio). Para ello ejcutamos el comando *back* para ir al menú principal:

<img src="img/empire_38.png" style="float:left">
<div style="clear:both"></div><br>

Desde el menú principal podemos acceder a los listeners:

<img src="img/empire_39.png" style="float:left">
<div style="clear:both"></div><br>

Para eliminarlo, usaremos el comando *kill* seguido del nombre del *listener* (servicio):

<img src="img/empire_40.png" style="float:left">
<div style="clear:both"></div><br>

Si ejecutamos el comando *list*, comprobaremos que se ha borrado:

<img src="img/empire_41.png" style="float:left">
<div style="clear:both"></div><br>

<a name="offline"></a>

### Ataques offline - Cracking de contraseñas

#### Identificar el algoritmo de hash usado para almacenar la contraseña

Existen programas que nos pueden ayudar a identificar el tipo de hash que se está usando para almacenar una contraseña. Esto puede ser muy útil para cuando más adelante intentemos crackearla. Vamos a ver dos programas:

  * hash-identifier
  * hashid
  
Yo recomiendo usar *hashid* porque, en mi experiencia, los resultados han sido más fiables.

**hash-identifier**

El programa *hash-identifier* nos puede ayudar a saber qué algoritmo de hashing, probablemente, se esté usando. No es 100% fiable pero es bastante aproximado. Para utilizarlo, primero copiamos el hash. En este primer ejemplo copiamos el hash del usuario msfadmin de la máquina metasploitable (distribución antigua):

<img src="img/credenciales_hashidentifier_1.png" style="float:left">
<div style="clear:both"></div><br>

después tecleamos *hash-identifier* y pulsamos intro. Pegamos el hash y pulsamos intro para que nos indique el tipo de hash:

<img src="img/credenciales_hashidentifier_2.png" style="float:left">
<div style="clear:both"></div><br>

en este caso, al ser una distribución antigua de GNU/Linux, vemos que se está usando MD5 que muy inseguro. Si probamos con la máquina de ubuntu server, veremos que nos indica que se esta usando *SHA256* cuando no es cierto porque usa *SHA512* pero, aunque menos seguro, es aproximado:

<img src="img/credenciales_hashidentifier_3.png" style="float:left">
<div style="clear:both"></div><br>

**hashid**

Es otro programa que podemos utilizar para saber qué tipo de hash se está usando. El modo de uso es practicamente igual. Listamos los hashes, tecleamos *hashid* y pulsamos intro:

<img src="img/credenciales_hashid_1.png" style="float:left">
<div style="clear:both"></div><br>

Ahora pegamos el hash del usuario *msfadmin* de la máquina metasploitable 2 y pulsamos intro para ver que es un MD5:

<img src="img/credenciales_hashid_2.png" style="float:left">
<div style="clear:both"></div><br>

A continuación hacemos lo mismo para el usuario *user* de la máquina ubuntu server para ver que es *SHA512*:

<img src="img/credenciales_hashid_3.png" style="float:left">
<div style="clear:both"></div><br>